## Neurons (using particle system)

This tutorial can be used to generate huge numbers of neurons, up to millions, with relatively little impact on the renderer's performance.

<a target="_blank" href="https://colab.research.google.com/github/VirtualBrainLab/urchin-examples/blob/colab/basics/neurons_probes.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Run the following block if using the notebook for the first time:
Urchin is a python package that needs to properly be installed before use the first time. Once installed within the environment, it only needs to be imported. If running locally, Urchin only needs to be installed once when creating the virtual environment. For more information about Urchin, check out the documentation on the VBL website [here](https://virtualbrainlab.org/urchin/installation_and_use.html).

In [ ]:
#Installing urchin
!pip install oursin -U

## Set up the Unity Renderer for Neuroscience and open the renderer webpage 
Ensure that popups are enabled BEFORE running the following code, to allow Urchin to properly open in a new window. You can also visit the renderer site once Urchin is imported at https://data.virtualbrainlab.org/Urchin/?ID=[INSERT ID OUTPUTTED BELOW]

In [ ]:
#Importing necessary libraries:
import oursin as urchin
urchin.setup()


## Neurons tutorial below

Neurons can be both created and manipulated either singularly or in plural form. The tutorial below displays both. The plural functions take in the list of neurons as the first parameter, followed by a list of the setting changed.

In [ ]:
#Creating 3 neurons
neurons_list = urchin.neurons.create(3)

In [ ]:
urchin.neurons.set_colors(neurons_list, ['#FFFFFF','#000000','#FF0000'])

In [ ]:
urchin.neurons.set_sizes(neurons_list, [5.0,4.0,3.0])

The singular functions below can also apply to neurons created within a list, by accessing them via index. 
ex: neurons_list[0].set_size(3.0)

In [ ]:
n1 = urchin.neurons.Neuron()

In [ ]:
n1.set_position([1000,5000,5000])

In [ ]:
n1.set_color('#00FF00')

In [ ]:
n1.set_size(4.0)

In [ ]:
n1.delete()

# Probes

Probes follow a very similar format to neurons, just have slightly different settings to adjust.

In [ ]:
probe_list = urchin.probes.create(3)

In [ ]:
urchin.probes.set_colors(probe_list, ['#FF00FF','#000000','#FF0000'])

In [ ]:
urchin.probes.set_angles(probe_list,[[90,45,45],[45,90,45],[45,45,90]])

In [ ]:
probe_list[0].set_color("#000000")

In [ ]:
probe_list[0].set_angle([90,45,45])

In [ ]:
probe_list[0].set_position([4000,4000,4000])

In [ ]:
probe_list[0].set_scale([0.07, 10, 0.02]) 